In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from credentials import user
from credentials import server
from credentials import password

In [2]:
connString = "postgres+psycopg2://"+user+":"+password+"@"+server+":5432/covid_data"
engine = create_engine(connString)

In [3]:
mask_use_df = pd.read_sql("SELECT * FROM mask_use;", con=connString)
counties_df = pd.read_sql("SELECT * FROM counties;", con=connString)

In [11]:
pop_df = pd.read_csv("../data/co-est2019-alldata.csv",encoding='latin-1')

In [26]:
pop_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2019,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019
0,40,3,6,1,0,Alabama,Alabama,4779736,4780125,4785437,...,1.917501,0.578434,1.186314,1.522549,0.563489,0.626357,0.745172,1.090366,1.773786,2.483744
1,50,3,6,1,1,Alabama,Autauga County,54571,54597,54773,...,4.847310,6.018182,-6.226119,-3.902226,1.970443,-1.712875,4.777171,0.849656,0.540916,4.560062
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183112,...,24.017829,16.641870,17.488579,22.751474,20.184334,17.725964,21.279291,22.398256,24.727215,24.380567
3,50,3,6,1,5,Alabama,Barbour County,27457,27455,27327,...,-5.690302,0.292676,-6.897817,-8.132185,-5.140431,-15.724575,-18.238016,-24.998528,-8.754922,-5.165664
4,50,3,6,1,7,Alabama,Bibb County,22915,22915,22870,...,1.385134,-4.998356,-3.787545,-5.797999,1.331144,1.329817,-0.708717,-3.234669,-6.857092,1.831952


In [12]:
mask_use_df.head()

,county_fips,never,rarely,sometimes,frequently,always
0,01001,0.053,0.074,0.134,0.295,0.444
1,01003,0.083,0.059,0.098,0.323,0.436
2,01005,0.067,0.121,0.120,0.201,0.491
3,01007,0.020,0.034,0.096,0.278,0.572
4,01009,0.053,0.114,0.180,0.194,0.459


In [13]:
counties_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0


In [14]:
counties_df.columns = ['date','county','state','county_fips','cases','deaths']

In [15]:
mask_counties = pd.merge(mask_use_df, counties_df, on='county_fips')

In [16]:
mask_counties.head()

,county_fips,never,rarely,sometimes,frequently,always,date,county,state,cases,deaths
0,01001,0.053,0.074,0.134,0.295,0.444,2020-03-24,Autauga,Alabama,1,0
1,01001,0.053,0.074,0.134,0.295,0.444,2020-03-25,Autauga,Alabama,4,0
2,01001,0.053,0.074,0.134,0.295,0.444,2020-03-26,Autauga,Alabama,6,0
3,01001,0.053,0.074,0.134,0.295,0.444,2020-03-27,Autauga,Alabama,6,0
4,01001,0.053,0.074,0.134,0.295,0.444,2020-03-28,Autauga,Alabama,6,0


In [17]:
mask_counties = mask_counties.drop_duplicates(subset=['county_fips'])
mask_counties.head()

,county_fips,never,rarely,sometimes,frequently,always,date,county,state,cases,deaths
0,01001,0.053,0.074,0.134,0.295,0.444,2020-03-24,Autauga,Alabama,1,0
235,01003,0.083,0.059,0.098,0.323,0.436,2020-03-14,Baldwin,Alabama,1,0
480,01005,0.067,0.121,0.120,0.201,0.491,2020-04-03,Barbour,Alabama,1,0
705,01007,0.020,0.034,0.096,0.278,0.572,2020-03-30,Bibb,Alabama,2,0
934,01009,0.053,0.114,0.180,0.194,0.459,2020-03-25,Blount,Alabama,1,0


In [18]:
good_complience = []
>>> for index, row in mask_counties.iterrows():
...     good_complience.append(row.frequently + row.always)

In [19]:
mask_counties['good_complience'] = good_complience

In [20]:
mask_counties = mask_counties.reset_index(drop=True)
mask_counties['good_rank'] = mask_counties['good_complience'].rank(ascending=False)
mask_counties.tail()

,county_fips,never,rarely,sometimes,frequently,always,date,county,state,cases,deaths,good_complience,good_rank
3128,56037,0.061,0.295,0.230,0.146,0.268,2020-03-24,Sweetwater,Wyoming,1,0,0.414,3107.0
3129,56039,0.095,0.157,0.160,0.247,0.340,2020-03-18,Teton,Wyoming,1,0,0.587,2581.0
3130,56041,0.098,0.278,0.154,0.207,0.264,2020-04-02,Uinta,Wyoming,1,0,0.471,3022.5
3131,56043,0.204,0.155,0.069,0.285,0.287,2020-03-27,Washakie,Wyoming,1,0,0.572,2654.0
3132,56045,0.142,0.129,0.148,0.207,0.374,2020-05-27,Weston,Wyoming,1,0,0.581,2615.0


In [21]:
davidson = mask_counties.loc[mask_counties['county'] == 'Davidson'].reset_index(drop=True)
davidson = davidson.loc[davidson['state'] == 'Tennessee'].reset_index(drop=True)

In [22]:
davidson.head()

,county_fips,never,rarely,sometimes,frequently,always,date,county,state,cases,deaths,good_complience,good_rank
0,47037,0.02,0.037,0.086,0.181,0.677,2020-03-08,Davidson,Tennessee,1,0,0.858,497.5


In [23]:
tennessee = mask_counties.loc[mask_counties['state'] == 'Tennessee'].reset_index(drop=True)
tennessee['good_rank'] = tennessee['good_complience'].rank(ascending=False)
tennessee.sort_values('good_complience', ascending=False)

,county_fips,never,rarely,sometimes,frequently,always,date,county,state,cases,deaths,good_complience,good_rank
23,47047,0.022,0.001,0.089,0.205,0.682,2020-03-21,Fayette,Tennessee,1,0,0.887,1.0
18,47037,0.020,0.037,0.086,0.181,0.677,2020-03-08,Davidson,Tennessee,1,0,0.858,2.0
78,47157,0.034,0.025,0.105,0.184,0.653,2020-03-08,Shelby,Tennessee,1,0,0.837,3.0
93,47187,0.019,0.073,0.091,0.212,0.604,2020-03-05,Williamson,Tennessee,1,0,0.816,4.0
85,47171,0.049,0.041,0.108,0.306,0.497,2020-03-26,Unicoi,Tennessee,1,0,0.803,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,47087,0.138,0.128,0.249,0.093,0.392,2020-04-02,Jackson,Tennessee,2,0,0.485,91.0
70,47141,0.160,0.164,0.207,0.099,0.369,2020-03-21,Putnam,Tennessee,5,0,0.468,92.0
7,47015,0.209,0.089,0.238,0.182,0.282,2020-03-26,Cannon,Tennessee,1,0,0.464,93.0
20,47041,0.157,0.152,0.231,0.132,0.328,2020-03-24,DeKalb,Tennessee,1,0,0.460,94.0


In Tennessee, Davidson County has the second highest percentage of respondents who always or frequently use masks.

In [24]:
tennessee['good_complience'].mean()

0.6367157894736842

Davidson County runs about 22% above the Tennessee mean in frequent or better mask usage...

In [25]:
mask_counties['good_complience'].mean()

0.7153916374082333

...and 14% above the US mean, ranking slightly better than 500th out of 3132 counties. Tennessee as a whole is 8% below the US mean.